# Procesamiento de Lenguaje Natural
Tecnicatura Universitaria en Inteligencia Artificial

Materia: IA4.2

**Trabajo Práctico 2**


**Docentes:**
- D'Alessandro, Ariel
- Geary, Alan
- Leon Cavallo, Andrea
- Manson, Juan Pablo


**Integrantes:**

| Apellido y Nombre | Legajo   |
| ----------------- | -------- |
| Aguirre, Fabián   | A-4516/1 |


Año: 2023

## Enunciado

# Pautas generales:

•	El trabajo deberá ser realizado individualmente.

•	Deberá informar cuál es la url del repositorio con el que van a trabajar y las definiciones de en qué problematicas quisieran solucionar con un sistema multiagente en el siguiente formulario:  https://docs.google.com/forms/d/e/1FAIpQLSdOZNGOzQ1gbf43caA4ygAbLx5tm5bU-s8RdfdftOzd_aXzhA/viewform?usp=pp_url

•	Se debe entregar un informe en el cual se incluya las justificaciones y un vínculo a los archivos que permitan reproducir el proyecto.

•	Temas deseables a cubrir en el tp

  * Recuperación de datos de bases de datos de grafos

  * Extracción de conocimiento de texto y posterior inserción en una base de datos de grafos

  * Agentes (estará cubierto en el Ejercicio 2)

# Ejercicio 1 - RAG

Crear un chatbot experto en un tema a elección, usando la técnica RAG (Retrieval Augmented Generation). Como fuentes de conocimiento se utilizarán al menos las siguientes fuentes:

•	Documentos de texto

•	Datos numéricos en formato tabular (por ej., Dataframes, CSV, sqlite, etc.)

•	Base de datos de grafos (Online o local)

El sistema debe poder llevar a cabo una conversación en lenguaje español. El usuario podrá hacer preguntas, que el chatbot intentará responder a partir de datos de algunas de sus fuentes. El asistente debe poder clasificar las preguntas, para saber qué fuentes de datos utilizar como contexto para generar una respuesta.

## Requerimientos generales

•	Realizar todo el proyecto en un entorno Google Colab

•	El conjunto de datos debe tener al menos 100 páginas de texto y un mínimo de 3 documentos.

•	Realizar split de textos usando Langchain (RecursiveTextSearch, u otros
métodos disponibles). Limpiar el texto según sea conveniente.

•	Realizar los embeddings que permitan vectorizar el texto y almacenarlo en una base de datos ChromaDB

•	Los modelos de embeddings y LLM para generación de texto son a elección


# Ejercicio 2 - Agentes

Realice una investigación respecto al estado del arte de las aplicaciones actuales de agentes inteligentes usando modelos LLM libres.

Plantee una problemática a solucionar con un sistema multiagente. Defina cada uno de los agentes involucrados en la tarea.

Realice un informe con los resultados de la investigación y con el esquema del sistema multiagente, no olvide incluir fuentes de información.

Opcional: Resolución con código de dicho escenario.



# Resolución

In [1]:
!pip install gdown
!pip install pymupdf
!pip install rdflib
!pip install pypdf langchain python-decouple
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install spacy
!python -m spacy download es_core_news_md
!pip install chromadb

#!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 26.4 MB/s eta 0:00:00
2023-12-22 18:58:54.658964: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register c

In [2]:
import gdown
import os
import shutil
import sys
import pandas as pd
import tensorflow_text
import tensorflow_hub as hub
import langchain
import chromadb
import fitz
import re
import requests
import spacy

from jinja2 import Template
from decouple import config
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rdflib import Graph, URIRef, Literal, Namespace
from sklearn.metrics.pairwise import cosine_similarity

Descarga de documentos

In [3]:
url = 'https://drive.google.com/drive/folders/1xLQnsyMX8vCWplS3RXqkVcdRdn42GhSA?usp=sharing'

gdown.download_folder(url, quiet=True, output='tp2-nlp')

carpeta_destino = 'documentos'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

carpeta_origen = 'tp2-nlp/tp2-nlp-documentos'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

shutil.rmtree(carpeta_origen)

carpeta_origen = 'tp2-nlp'
carpeta_destino = '/content/'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

shutil.rmtree(carpeta_origen)

sys.path.append('/content/')

print("Archivos descargados con éxito.")

Archivos descargados con éxito.


In [4]:
#Creación de la base con CrhomaDB
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

client = chromadb.Client()
collection = client.get_or_create_collection("all-my-documents")

pdf_directory = "documentos"

pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]

textos = []
ids_textos = []
fuentes = []

def clean_text(text):
    cleaned_text = text.lower()
    cleaned_text = re.sub(r'[^\w\s.,]', '', cleaned_text)
    return cleaned_text

splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=10)

for i, pdf_file in enumerate(pdf_files, start=1):
    pdf_path = os.path.join(pdf_directory, pdf_file)

    with fitz.open(pdf_path) as doc:
        print(pdf_path)
        text = ""
        for page_num in range(doc.page_count):
            page = doc[page_num]
            text += page.get_text()

    split_texts = splitter.split_text(text)

    clean_texts = [clean_text(sentence) for sentence in split_texts]

    textos.extend(clean_texts)
    ids_textos.extend([f"doc{i}_{j}" for j in range(1, len(clean_texts) + 1)])
    fuentes.extend([f"fuente{i}" for _ in range(len(clean_texts))])

embeddings = embed(textos).numpy().tolist()

collection.add(
    documents=textos,
    metadatas=[{"source": fuente} for fuente in fuentes],
    ids=ids_textos,
    embeddings=embeddings)
#client.delete_collection('all-my-documents')

documentos/etica_y_trasplante.pdf
documentos/alimentacion-basada-en-plantas.pdf
documentos/investigacion-sodio.pdf
documentos/documento-entornos-escolares-saludables.pdf


In [ ]:
#Creación del grafo
nlp = spacy.load("es_core_news_md")
spl = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)

def clean_text_pdf(text):
    cleaned_text = text.lower()
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    return cleaned_text

def read_pdf(path):
    with fitz.open(path) as doc:
        text = ""
        for page_num in range(doc.page_count):
            page = doc[page_num]
            text += page.get_text()
    split_texts = spl.split_text(text)
    clean_texts = [clean_text_pdf(sentence) for sentence in split_texts]
    return clean_texts

def extract_entities_relations(doc):
    subject = ""
    obj = ""
    relation = ""
    prep_obj = ""

    for token in doc:
        if "subj" in token.dep_:
            subject = token.text
        if "obj" in token.dep_:
            obj = token.text
        if token.dep_ == "ROOT":
            relation = token.text
            for child in token.children:
                if child.dep_ == "nmod":
                    prep_obj = child.text
        if token.dep_ == "prep":
            for child in token.children:
                prep_obj = child.text
    if prep_obj:
        obj = prep_obj
    return (subject, obj), [relation]

# Crear un grafo RDF
g = Graph()
n = Namespace("http://chagas.org/")

chagas_pdf_path = "recomendaciones-chagas.pdf"
text_pdf_chagas = read_pdf(chagas_pdf_path)

for sentence in text_pdf_chagas:
    doc = nlp(sentence)
    entities, relations = extract_entities_relations(doc)
    print(entities, relations)
    if entities[0] and entities[1] and relations:
        subject_name, object_name = entities
        relation_name = relations[0]
        subject = URIRef(n + subject_name)
        predicate = URIRef(n + relation_name)
        obj = URIRef(n + object_name)
        g.add((subject, predicate, obj))

rdf_output = g.serialize(format='xml')

with open("graph.rdf", "w") as file:
    file.write(rdf_output)


In [18]:
#CHATBOT
def read_csv(path_csv):
    csv_path = path_csv
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    context_csv = ""
    for _, row in df.iterrows():
        context_csv += f"{row['jurisdiccion']} tiene una tasa de VIH de {row['tasa_vih']}\n"
    return context_csv


def read_graph(path_graph, query_user):
    nlp = spacy.load("es_core_news_md")

    n = Namespace("http://chagas.org/")

    g = Graph()
    g.parse(path_graph, format="xml")

    def cosine_similarity_entities(query_embeddings, entity_embedding):
        similarity_score = cosine_similarity([query_embeddings], [entity_embedding])[0][0]
        return similarity_score

    def are_entities_in_graph(query_embedding_0, query_embedding_1):
        max_similarity_score = 0.0
        matching_entity = None

        for node in g.all_nodes():
            node_name = node.toPython()

            node_embeddings = embed([node_name]).numpy().tolist()[0]

            similarity_score_0 = cosine_similarity_entities(query_embedding_0, node_embeddings)
            similarity_score_1 = cosine_similarity_entities(query_embedding_1, node_embeddings)

            if max(similarity_score_0, similarity_score_1) > max_similarity_score:
                max_similarity_score = max(similarity_score_0, similarity_score_1)
                matching_entity = node_name

            return  max_similarity_score

    entities, _ = extract_entities_relations(nlp(query_user))

    query_embedding_0 = embed([n + entities[0]]).numpy().tolist()[0]
    query_embedding_1 = embed([n + entities[1]]).numpy().tolist()[0]

    similarity_graph = are_entities_in_graph(query_embedding_0, query_embedding_1)
    context_graph = ""
    for s, p, o in g:
        subject_name = s.split(n)[-1]
        predicate_name = p.split(n)[-1]
        object_name = o.split(n)[-1]
        context_graph += f"{subject_name} {predicate_name} {object_name}.\n"

    return similarity_graph, context_graph


def query_chromadb(query_embed):
    results = collection.query(
        query_embeddings=query_embed,
        n_results=20
    )
    return results


def classifier(query_str: str) -> str:
    query_embeddings = embed([query_str]).numpy().tolist()
    reference_text_csv = "VIH"
    reference_embedding_csv = embed([reference_text_csv]).numpy().tolist()
    max_cosine_similarity_csv = -1000

    for  text_segment_csv in query_str.split():
        segment_embedding_csv = embed([text_segment_csv]).numpy().tolist()
        similarity_score_csv = cosine_similarity(segment_embedding_csv, reference_embedding_csv)[0][0]

        if similarity_score_csv > max_cosine_similarity_csv:
           max_cosine_similarity_csv = similarity_score_csv

    path_csv_vih = "tasa_vih_jurid.csv"
    context_csv = read_csv(path_csv_vih)

    documents = query_chromadb(query_embeddings)
    diference_min_chromadb = min(abs(x - 1) for x in documents["distances"][0])
    context_chroma = "\n".join([f"{doc}" for documents_lista in documents["documents"] for doc in documents_lista])
    similarity_chroma = 1 - diference_min_chromadb


    path_graph_chagas = "graph.rdf"
    similarity_graph, context_graph = read_graph(path_graph_chagas,query_str)

    if similarity_chroma > max_cosine_similarity_csv and similarity_chroma > similarity_graph:
        return context_chroma
    elif similarity_graph > similarity_chroma and similarity_graph > max_cosine_similarity_csv:
        return context_graph
    else:
        return context_csv


def zephyr_chat_template(messages, add_generation_prompt=True):

    template_str = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    template = Template(template_str)
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)




def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    try:

        api_key = config('HUGGINGFACE_TOKEN')

        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        headers = {"Authorization": f"Bearer {api_key}"}

        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        response = requests.post(api_url, headers=headers, json=data)

        respuesta = response.json()[0]["generated_text"][len(prompt):]

        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")


def prepare_prompt(query_str, context_prompt):

    TEXT_QA_PROMPT_TMPL = (
        "La información de contexto es la siguiente:\n"
        "---------------------\n"
        "{context_prompt}\n"
        "---------------------\n"
        "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n"
        "Pregunta: {query_str}\n"
        "Respuesta: "
    )

    messages = [
        {
            "role": "system",
            "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
        },
        {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_prompt=context_prompt, query_str=query_str)},
    ]

    final_prompt = zephyr_chat_template(messages)
    return final_prompt

print('**BIENVENIDOS AL CHATBOT DE SALUD Y ALIMENTACIÓN**\n')
print('Las preguntas puede estar relacionadas con los siguiente temas: \n')
print('-Alimentación basada en plantas\n')
print('-Entornos escolares saludables\n')
print('-Ética y transplantes\n')
print('-Sodio en la alimentación\n')
print('-Tasa de VIH por provincias\n')

while True:
    user_query = input("Ingrese su consulta (o 'salir' para salir): ")
    if user_query.lower() == 'salir':
        break

    context_final = classifier(user_query)
    final_prompt = prepare_prompt(user_query, context_final)
    print('Respuesta:')
    print(generate_answer(final_prompt))
    print('-------------------------------------------------------')

#Ejemplos de preguntas
#¿Cuál es la tasa de VIH de Neuquen?
#¿Cómo es el procedimiento para una transplante de organos?
#¿Qué puede producir en el cuerpo el exceso de consumo de sodio en los alimentos?
#¿Cuáles son los requerimientos para personas con alimentación vegetariana?
#En pocas palabras ¿Cómo controlar el vector del chagas?

**BIENVENIDOS AL CHATBOT DE SALUD Y ALIMENTACIÓN**

Las preguntas puede estar relacionadas con los siguiente temas: 

-Alimentación basada en plantas

-Entornos escolares saludables

-Ética y transplantes

-Sodio en la alimentación

-Tasa de VIH por provincias

Ingrese su consulta (o 'salir' para salir): salir
